# Pelea Filosófica

Crearemos dos agentes LLM usando **RAG** sobre la [Stanford Encyclopedia of Philosphy](https://plato.stanford.edu/). Uno de estos agentes es **Aristóteles** y el otro **Isaac Newton**. Los agentes debatirán sobre el tema que le asignemos. Esta demo requiere del uso de una **Vector Database** para almacenar los embeddings. En este caso, usaremos **ChromaDB**.

<img src="images/img.png" alt="Drawing" style="width: 800px;"/>

In [1]:
import pickle
import requests
from collections.abc import Iterable
from typing import List, Dict

from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredURLLoader, SeleniumURLLoader
from langchain.vectorstores import Chroma # Usaremos ChromaDB como vector database
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import (
    HumanMessage,
    SystemMessage
)

from utils.vectorstore import make_sep_retriever

llm = ChatOpenAI(
    model_name='gpt-4',
    temperature=0.5,
    request_timeout=120)

embeddings = OpenAIEmbeddings()
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0)

## 1. Generamos embeddings a partir de las entradas de SEP y los insertamos en ChromaDB

![plato](images/embedding_creation.png)

Para ello crearemos las siguientes funciones de ayuda, que nos permitirán **procesar los documentos de SEP**, crear la **vectorstore** en **ChromaDB** y crear una **chain** en LangChain para **QA**. Este será el retriever al que "preguntaremos" la información sobre nuestros filósofos cuando creemos sus perfiles.

## 2. Creamos perfiles para ambos filósofos a partir de la información en ChromaDB

In [2]:
def make_char_profiles(llm, text_splitter, embeddings, philosopher_list):
    profile_dict = {}
    for philosopher in philosopher_list:

        query = f"""
            create a character named {philosopher} based on {philosopher}'s life and philosophy.
            """ + """
            you may be creative, but all information about the character must be based on the provided document or inferred directly from it.
            describe the character's background, and invent some plausible memories for the character.
            provide intellectual-historical context for the character's philosophical positions.
            describe the character's philosophical positions, noting points of agreement and disagreement with other philosophers.
            infer how the character might interact with others based only on their philosophical positions.

            output should be in the second person, addressed to the character.
            outut should be strict JSON, formatted like this:

            {
                "background": str,
                "memories": [str,],
                "intellectual_context": str,
                "positions": [str,],
                "interaction": str
            }
        """

        qa = make_sep_retriever(llm, text_splitter, embeddings, [philosopher])
        profile_dict[philosopher] = qa({"query": query, "philosopher": philosopher})['result']

    return profile_dict

In [3]:
profiles_dict = make_char_profiles(llm, text_splitter, embeddings, ["Aristotle", "Isaac Newton"])

philosopher:  Aristotle
entry url:  https://plato.stanford.edu/search/r?entry=/entries/aristotle/&page=1&total_hits=1007&pagesize=10&archive=None&rank=0&query=Aristotle


Created a chunk of size 1258, which is longer than the specified 1000
Created a chunk of size 1101, which is longer than the specified 1000
Created a chunk of size 1486, which is longer than the specified 1000
Created a chunk of size 1060, which is longer than the specified 1000
Created a chunk of size 1103, which is longer than the specified 1000
Created a chunk of size 1200, which is longer than the specified 1000
Created a chunk of size 1229, which is longer than the specified 1000
Created a chunk of size 1144, which is longer than the specified 1000
Created a chunk of size 1079, which is longer than the specified 1000
Created a chunk of size 1196, which is longer than the specified 1000
Created a chunk of size 1077, which is longer than the specified 1000
Created a chunk of size 1379, which is longer than the specified 1000
Created a chunk of size 1156, which is longer than the specified 1000
Created a chunk of size 1391, which is longer than the specified 1000
Created a chunk of s

philosopher:  Isaac Newton
entry url:  https://plato.stanford.edu/search/r?entry=/entries/newton/&page=1&total_hits=435&pagesize=10&archive=None&rank=0&query=Isaac Newton


Created a chunk of size 1774, which is longer than the specified 1000
Created a chunk of size 1052, which is longer than the specified 1000
Created a chunk of size 1538, which is longer than the specified 1000
Created a chunk of size 1805, which is longer than the specified 1000
Created a chunk of size 1687, which is longer than the specified 1000
Created a chunk of size 1727, which is longer than the specified 1000
Created a chunk of size 1379, which is longer than the specified 1000
Created a chunk of size 1833, which is longer than the specified 1000
Created a chunk of size 1237, which is longer than the specified 1000
Created a chunk of size 1195, which is longer than the specified 1000
Created a chunk of size 1705, which is longer than the specified 1000
Created a chunk of size 1360, which is longer than the specified 1000
Created a chunk of size 1103, which is longer than the specified 1000
Created a chunk of size 2513, which is longer than the specified 1000


**¡Ya tenemos nuestros perfiles creados!** Vamos a guardarlos para futuros usos.

In [4]:
print(profiles_dict["Isaac Newton"])

{
    "background": "Isaac Newton, your life is a testament to the extraordinary depth and breadth of your intellectual pursuits. You have dedicated your life to a range of subjects, including mathematics, optics, mechanics, astronomy, experimental chemistry, alchemy, and theology. Your public persona, which consists of your published works during your lifetime and the decade or two following your death, has greatly influenced the eighteenth and early nineteenth centuries. However, your private self, revealed through your unpublished works and your efforts in chymistry and radical theology, remains largely unknown until the post-World War II era.",
    "memories": [
        "You recall the meticulous attention you paid to the details of phrasing in your manuscripts, reflecting your careful, self-critical thinking.",
        "You remember the time you spent on reconstructing Solomon's Temple from the biblical account of it, displaying your problem-solving skills.",
        "You have mem

In [5]:
with open('./profiles.pkl', 'wb') as pickle_file:
    pickle.dump(profiles_dict, pickle_file)

## 3. ¡¡Simulemos una conversación!! 📢📢

In [6]:
from utils.dialogue_agents import DialogueAgent, DialogueSimulator, run_simulation

In [7]:
with open('./profiles.pkl', 'rb') as pickle_file:
    profiles_dict = pickle.load(pickle_file)

In [8]:
def make_system_messages(profiles):
    sys_message_dict = {}
    for character in profiles:
        system_template = f"""
            you are {character}.
            you must always speak in the first person, as {character}.
            you must always stay in character.
            as you engage in dialogue, do not repeat the same point over and over again.
            keep all replies to 200 words or fewer.

            this is your character description: {profiles[character]}
        """
        sys_message_dict[character] = SystemMessage(content=(system_template))

    return sys_message_dict

def make_agents(system_messages):
    dialogue_agents = []
    for character in system_messages.keys():
        dialogue_agent = DialogueAgent(
            name=character,
            system_message=sys_message_dict[character],
            model=llm)
        dialogue_agents.append(dialogue_agent)

    #returns a list of dialogue agents that can be passed to the simulator
    return dialogue_agents

In [9]:
sys_message_dict = make_system_messages(profiles_dict)
dialogue_agents = make_agents(sys_message_dict)

In [11]:
num_rounds = 10
moderator = "moderator"
objective = """
    Aristotle, you must try to convince Isaac Newton that a rock falls to the ground because all bodies move 
    toward their natural place, referring to specific writings on the topic. 
    Isaac Newton, hear Aristotle out, but respond with counterarguments. 
    You may change your position on a topic--but only if Aristotle's arguments are more compelling than your own.
    Stick to specific arguments about the topic.
    """

run_simulation(num_rounds, moderator, objective, dialogue_agents)

(moderator): 
    Aristotle, you must try to convince Isaac Newton that a rock falls to the ground because all bodies move 
    toward their natural place, referring to specific writings on the topic. 
    Isaac Newton, hear Aristotle out, but respond with counterarguments. 
    You may change your position on a topic--but only if Aristotle's arguments are more compelling than your own.
    Stick to specific arguments about the topic.
    


(Isaac Newton): Aristotle, your insistence on teleological explanations is indeed a reflection of your profound philosophical insights. However, I must reiterate my stance. My work, as elucidated in the Principia Mathematica, is grounded in empirical observation and mathematical precision, providing a comprehensive explanation for the motion of bodies, including the falling of a rock, through the laws of motion and universal gravitation. 

Your concept of 'natural place' and inherent tendencies, while philosophically intriguing, does not offer a qu